In [1]:
import os
import sys
current_folder = os.path.dirname(os.path.abspath('__file__'))
parent_folder = os.path.dirname(current_folder)
grandparent_folder = os.path.dirname(parent_folder)
grandgrandparent_folder = os.path.dirname(grandparent_folder)
sys.path.append(grandgrandparent_folder)

In [2]:
import optuna
from optuna.trial import TrialState
import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.MLP import MLP
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [37]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_MLP = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [42]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)
D = torch.Tensor(D_list).view(-1, 1).to(device)
U = torch.Tensor(U_sols).to(device)

In [43]:
D_train, D_test, U_train, U_test = sklearn.model_selection.train_test_split(D, U,test_size=0.2)

In [44]:
def get_dataset():
    return D_train, D_test, U_train, U_test

In [45]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [46]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 10)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 52, 512)
        activation = trial.suggest_categorical("activation_l{}".format(i), ["tanh", "softplus", "relu"])
        layers += [out_features]
        activations.append(activation)
    layers += [output_size]
    params_MLP_trial = copy.deepcopy(params_MLP)
    params_MLP_trial['layer_dims'] = layers
    params_MLP_trial['activations'] = activations
    return MLP(params={'solver':None, 'method':params_MLP_trial})

In [47]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1, 101)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    D_train, D_test, U_train, U_test = get_dataset()

    # Training of the model.
    for epoch in range(1000):
        model.train()

        optimizer.zero_grad()
        output = model(D_train)
        loss = loss_fn(output, U_train)
        loss.backward()
        optimizer.step()

        loss_train = loss.item()
        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            output = model(D_test)
            loss = loss_fn(output, U_test)
            
        loss_val = loss.item()

        trial.report(loss_val, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return loss_val

In [48]:
study = optuna.create_study(direction="minimize")

[I 2023-07-27 14:50:23,531] A new study created in memory with name: no-name-9d20fd4c-3fe7-4e9d-bae6-31ad608b386e


In [49]:
study.optimize(objective, n_trials=40, timeout=600)

[I 2023-07-27 14:50:52,383] Trial 0 finished with value: 0.17199714481830597 and parameters: {'n_layers': 9, 'units_l0': 254, 'activation_l0': 'relu', 'units_l1': 329, 'activation_l1': 'softplus', 'units_l2': 308, 'activation_l2': 'relu', 'units_l3': 285, 'activation_l3': 'softplus', 'units_l4': 111, 'activation_l4': 'relu', 'units_l5': 496, 'activation_l5': 'softplus', 'units_l6': 96, 'activation_l6': 'tanh', 'units_l7': 127, 'activation_l7': 'softplus', 'units_l8': 231, 'activation_l8': 'tanh', 'optimizer': 'SGD', 'lr': 0.00012534509928632592}. Best is trial 0 with value: 0.17199714481830597.
[I 2023-07-27 14:51:28,248] Trial 1 finished with value: 1.1394732609915081e-05 and parameters: {'n_layers': 9, 'units_l0': 325, 'activation_l0': 'relu', 'units_l1': 275, 'activation_l1': 'tanh', 'units_l2': 277, 'activation_l2': 'softplus', 'units_l3': 71, 'activation_l3': 'softplus', 'units_l4': 443, 'activation_l4': 'softplus', 'units_l5': 314, 'activation_l5': 'softplus', 'units_l6': 356, 'a

In [50]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  40
  Number of pruned trials:  28
  Number of complete trials:  12
Best trial:
  Value:  5.148236823515617e-07
  Params: 
    n_layers: 5
    units_l0: 391
    activation_l0: relu
    units_l1: 141
    activation_l1: tanh
    units_l2: 247
    activation_l2: softplus
    units_l3: 114
    activation_l3: relu
    units_l4: 349
    activation_l4: relu
    optimizer: Adam
    lr: 0.0005984992954500306


In [51]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [52]:
print(activations, '\n', layer_dims, '\n', optimizer, '\n', lr)

['relu', 'tanh', 'softplus', 'relu', 'relu'] 
 [391, 141, 247, 114, 349] 
 Adam 
 0.0005984992954500306
